# ISSS609-Text Analytics and Applications Project

There will be a list of workbooks for the purpose of this project.

1) Data_Preparation_and_Topic_Modelling

2) Abstractive_Summarisation

3) Extractive_Summarisation_Average_Score

4) Extractive_Summarisation_TFIDF

5) UI_Preparation

### This workbook will cover (2)  Abstractive Summarisation

In this notebook We will write summaries with the help of my Seq2Seq model in Summarizer.py.

#### Raw Data are not provided as it is too huge. Can be downloaded from the following link:
https://www.kaggle.com/snapcrack/all-the-news


In [1]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from collections import Counter

import Summarizer
import summarizer_data_utils
import summarizer_model_utils

W0331 12:00:03.322537  3056 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
print(tf.__version__)

1.13.1


### Reading and exploring

In [3]:
# the dataset consists of 3 .csv files. we will concatenate them.
data = pd.read_csv('Raw Data/articles1.csv',
                   encoding='utf-8')
data1 = pd.read_csv('Raw Data/articles2.csv',
                    encoding='utf-8')
data2 = pd.read_csv('Raw Data/articles3.csv',
                    encoding='utf-8')


In [4]:
data = pd.concat([data, data1, data2])
data.shape

(142570, 10)

In [5]:
# we are only going to use title and content.
data.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [6]:
Counter(data.publication)

Counter({'New York Times': 7803,
         'Breitbart': 23781,
         'CNN': 11488,
         'Business Insider': 6757,
         'Atlantic': 7179,
         'Fox News': 4354,
         'Talking Points Memo': 5214,
         'Buzzfeed News': 4854,
         'National Review': 6203,
         'New York Post': 17493,
         'Guardian': 8681,
         'NPR': 11992,
         'Reuters': 10710,
         'Vox': 4947,
         'Washington Post': 11114})

In [7]:
# Use only 4 chosen publications

P1 = data["publication"]== "Washington Post"
P2 = data["publication"]== "Reuters"
P3 = data["publication"]== "New York Times"
P4 = data["publication"]== "Guardian"
data = data[P1|P2|P3|P4]

In [8]:
data.reset_index(drop=True, inplace=True)
del data["Unnamed: 0"]

In [9]:
data.rename(index = str, columns = {'title':'Summary', 'content':'Text'}, inplace = True)
data = data[['Summary', 'Text']]

In [10]:
data = data[['Summary', 'Text']]
data.head()

,Summary,Text
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."


In [11]:
# Save as a copy for future reference

data.to_csv('summary-text.csv')

In [12]:
# Check

for x in data.Summary[:10]:
    print(x)

House Republicans Fret About Winning Their Health Care Suit - The New York Times
Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times
Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial Bias, Dies at 106 - The New York Times
Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times
Kim Jong-un Says North Korea Is Preparing to Test Long-Range Missile - The New York Times
Sick With a Cold, Queen Elizabeth Misses New Year’s Service - The New York Times
Taiwan’s President Accuses China of Renewed Intimidation - The New York Times
After ‘The Biggest Loser,’ Their Bodies Fought to Regain Weight - The New York Times
First, a Mixtape. Then a Romance. - The New York Times
Calling on Angels While Enduring the Trials of Job - The New York Times


In [13]:
data.Text[0]

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [14]:
# Check na AND fill it

data.isna().sum()

Summary    1
Text       0
dtype: int64

In [15]:
data = data.dropna()

In [16]:
# try to pick some of the data. 
len_summaries = [len(summary) for i, summary in enumerate(data.Summary)]
len_texts = [len(text) for text in data.Text]

In [17]:
len_summaries_counted = Counter(len_summaries).most_common()
len_texts_counted = Counter(len_texts).most_common()
len_summaries_counted[:10], len_texts_counted[:10]

([(65, 1118),
  (64, 1100),
  (68, 1095),
  (70, 1084),
  (67, 1076),
  (69, 1069),
  (71, 1065),
  (72, 1063),
  (74, 1062),
  (63, 1061)],
 [(2878, 19),
  (5387, 18),
  (4886, 17),
  (3385, 17),
  (2969, 17),
  (3059, 17),
  (2973, 16),
  (3089, 15),
  (4426, 15),
  (4844, 15)])

In [27]:
# Use shorter text first

indices = [ind for ind, text in enumerate(data.Text) if 500 < len(text) < 1500]

In [28]:
texts_unprocessed = data.Text[indices]
summaries_unprocessed = data.Summary[indices]

In [29]:
len(indices), len(texts_unprocessed), len(summaries_unprocessed)

(808, 808, 808)

In [30]:
# Articles from NYT seem to have those endings, therefore
# we will remove those, as that is not relevant

to_remove = ['- The New York Times']

summaries_unprocessed_clean = []
texts_unprocessed_clean = []

removed = 0
append = True
for sentence in summaries_unprocessed:
    append = True
    for r in to_remove:
        if sentence.endswith(r):
            sentence = sentence.replace(r, '.')
            summaries_unprocessed_clean.append(sentence.replace(r, '.'))
            removed+=1
            append = False
            break
            
    if append:
        summaries_unprocessed_clean.append(sentence)
       


In [31]:
len(summaries_unprocessed_clean), len(texts_unprocessed)

(808, 808)

### Clean and prepare the data

In [32]:
# preprocess the texts and summaries.
# We have the option to keep_most or not. in this case we do not want 'to keep most', i.e. we will only keep
# letters and numbers. 
# (to improve the model, this preprocessing step should be refined)
processed_texts, processed_summaries, words_counted = summarizer_data_utils.preprocess_texts_and_summaries(
    texts_unprocessed,
    summaries_unprocessed_clean,
    keep_most=False)

Processing Time:  1.7467880249023438


In [33]:
# some of the texts are empty remove those. 
processed_texts_clean = []
processed_summaries_clean = []

for t, s in zip(processed_texts, processed_summaries):
    if t != [] and s != []:
        processed_texts_clean.append(t)
        processed_summaries_clean.append(s)

### Create lookup dicts

We cannot feed our network actual words, but numbers. So we first have to create our lookup dicts, where each words gets and int value (high or low, depending on its frequency in our corpus). Those help us to later convert the texts into numbers.

We also add special tokens. EndOfSentence and StartOfSentence are crucial for the Seq2Seq model we later use.
Pad token, because all summaries and texts in a batch need to have the same length, pad token helps us do that.

So we need 2 lookup dicts:
 - From word to index 
 - from index to word. 

In [53]:
# Create lookup dicts.
# most oft the words only appear only once. 
# min_occureces set to 2 reduces our vocabulary by more than half.
specials = ["<EOS>", "<SOS>", "<PAD>", "<UNK>"]
word2ind, ind2word,  missing_words = summarizer_data_utils.create_word_inds_dicts(words_counted,
                                                                                  specials = specials,
                                                                                  min_occurences = 2)
print(len(word2ind), len(ind2word), len(missing_words))


9232 9232 7699


### Pretrained embeddings

Optionally we can use pretrained word embeddings. Those have proved to increase training speed and accuracy.
Here I used two different options. Either we use glove embeddings or embeddings from tf_hub.
The ones from tf_hub worked better.

In [46]:
# The embeddings from tf.hub. 
# embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim128/1")
embed = hub.Module("https://tfhub.dev/google/Wiki-words-250/1")
emb = embed([key for key in word2ind.keys()])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    embedding = sess.run(emb)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0331 17:35:34.329401  3056 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [47]:
embedding.shape

(9232, 250)

In [48]:
np.save('./tf_hub_embedding_headlines.npy', embedding)

### Convert text and summaries
As I said before we cannot feed the words directly to our network, we have to convert them to numbers first of all. This is what we do here. And we also append the SOS and EOS tokens.

In [54]:
# converts words in texts and summaries to indices
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(processed_texts_clean,
                                                                                word2ind,
                                                                                eos = False)

In [55]:
converted_summaries, unknown_words_in_summaries = summarizer_data_utils.convert_to_inds(processed_summaries_clean,
                                                                                        word2ind,
                                                                                        eos = True,
                                                                                        sos = True)

In [56]:
# seems to have worked well. 
print(summarizer_data_utils.convert_inds_to_text(converted_texts[0], ind2word))
print(summarizer_data_utils.convert_inds_to_text(converted_summaries[0], ind2word))


['london', 'queen', 'elizabeth', 'ii', 'who', 'has', 'been', 'battling', 'a', 'cold', 'for', 'more', 'than', 'a', 'week', 'missed', 'a', 'new', 'year', 's', 'day', 'church', 'service', 'at', 'her', 'country', 'estate', 'in', 'sandringham', 'buckingham', 'palace', 'said', 'on', 'sunday', 'a', 'week', 'earlier', 'the', 'queen', 'who', 'is', '90', 'missed', 'a', 'christmas', 'day', 'church', 'service', 'for', 'the', 'first', 'time', 'since', '1988', 'because', 'of', 'the', 'illness', 'the', 'queen', 'does', 'not', 'yet', 'feel', 'ready', 'to', 'attend', 'church', 'as', 'she', 'is', 'still', 'recuperating', 'from', 'a', 'heavy', 'cold', 'the', 'palace', 'said', 'in', 'a', 'statement', 'the', 'queen', 's', 'husband', 'prince', 'philip', 'who', 'had', 'also', 'been', 'ill', 'was', 'well', 'enough', 'to', 'attend', 'both', 'services', 'in', 'the', 'church', 'at', 'sandringham', 'which', 'is', 'in', 'norfolk', 'on', 'the', 'east', 'coast', 'of', 'england', 'the', 'queen', 'who', '<UNK>', 'to',

## The model

Now we can build and train our model. First we define the hyperparameters we want to use. Then we create our Summarizer and call the function .build_graph(), which as the name suggests, builds the computation graph. 
Then we can train the model using .train()

After training we can try our model using .infer()

### Training
Unfortunately I do not have the resources to find the perfect (or right) hyperparameters, but these do pretty well. 

I trained the model for about 40 epochs. the training loss, as well as the validation loss were both still declining.
I chose to use 90% of the data as trainign set and 10% as validation set. We could have also used sklearn's train_test_split here. 

In [57]:
# model hyperparameters
num_layers_encoder = 4
num_layers_decoder = 4
rnn_size_encoder = 300
rnn_size_decoder = 300

batch_size = 32
epochs = 100
clip = 5
keep_probability = 0.8
learning_rate = 0.0005
max_lr=0.005
learning_rate_decay_steps = 100
learning_rate_decay = 0.90


pretrained_embeddings_path = './tf_hub_embedding_headlines.npy'
summary_dir = os.path.join('./tensorboard/headlines')

use_cyclic_lr = True
inference_targets=True


In [58]:
# build graph and train the model 
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   save_path='./models/headlines/my_model',
                                   mode='TRAIN',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   batch_size = batch_size,
                                   clip = clip,
                                   keep_probability = keep_probability,
                                   learning_rate = learning_rate,
                                   max_lr=max_lr,
                                   learning_rate_decay_steps = learning_rate_decay_steps,
                                   learning_rate_decay = learning_rate_decay,
                                   epochs = epochs,
                                   pretrained_embeddings_path = pretrained_embeddings_path,
                                   use_cyclic_lr = use_cyclic_lr,)
#                                    summary_dir = summary_dir)           

summarizer.build_graph()
summarizer.train(converted_texts, 
                 converted_summaries)


Loaded pretrained embeddings.
Graph built.
-------------------- Epoch 0 of 100 --------------------
Iteration: 0 of 25	train_loss: 9.1303
Iteration: 2 of 25	train_loss: 9.1252
Iteration: 4 of 25	train_loss: 9.1036
Iteration: 6 of 25	train_loss: 8.9791
Iteration: 8 of 25	train_loss: 8.6281
Iteration: 10 of 25	train_loss: 8.1960
Iteration: 12 of 25	train_loss: 7.7302
Iteration: 14 of 25	train_loss: 7.3751
Iteration: 16 of 25	train_loss: 7.1056
Iteration: 18 of 25	train_loss: 7.2893
Iteration: 20 of 25	train_loss: 7.1196
Iteration: 22 of 25	train_loss: 7.4651
Iteration: 24 of 25	train_loss: 7.1606
Iteration: 25 of 25	train_loss: 7.0187
Average Score for this Epoch: 7.9821085929870605
--- new best score ---


-------------------- Epoch 1 of 100 --------------------
Iteration: 0 of 25	train_loss: 6.6843
Iteration: 2 of 25	train_loss: 6.6413
Iteration: 4 of 25	train_loss: 6.6706
Iteration: 6 of 25	train_loss: 6.5544
Iteration: 8 of 25	train_loss: 6.7101
Iteration: 10 of 25	train_loss: 6.5362

Iteration: 10 of 25	train_loss: 6.0587
Iteration: 12 of 25	train_loss: 6.0859
Iteration: 14 of 25	train_loss: 6.1222
Iteration: 16 of 25	train_loss: 6.1663
Iteration: 18 of 25	train_loss: 6.0932
Iteration: 20 of 25	train_loss: 6.1666
Iteration: 22 of 25	train_loss: 6.1991
Iteration: 24 of 25	train_loss: 6.0566
Iteration: 25 of 25	train_loss: 5.9526
Average Score for this Epoch: 6.012979984283447
-------------------- Epoch 13 of 100 --------------------
Iteration: 0 of 25	train_loss: 5.6501
Iteration: 2 of 25	train_loss: 5.8088
Iteration: 4 of 25	train_loss: 5.7223
Iteration: 6 of 25	train_loss: 6.0127
Iteration: 8 of 25	train_loss: 5.8456
Iteration: 10 of 25	train_loss: 5.7570
Iteration: 12 of 25	train_loss: 5.8494
Iteration: 14 of 25	train_loss: 6.0190
Iteration: 16 of 25	train_loss: 5.8928
Iteration: 18 of 25	train_loss: 5.8953
Iteration: 20 of 25	train_loss: 5.9947
Iteration: 22 of 25	train_loss: 6.1389
Iteration: 24 of 25	train_loss: 5.8444
Iteration: 25 of 25	train_loss: 5.7234
Av

Iteration: 24 of 25	train_loss: 5.6511
Iteration: 25 of 25	train_loss: 5.5139
Average Score for this Epoch: 5.5401177406311035
-------------------- Epoch 25 of 100 --------------------
Iteration: 0 of 25	train_loss: 5.4378
Iteration: 2 of 25	train_loss: 5.4972
Iteration: 4 of 25	train_loss: 5.4451
Iteration: 6 of 25	train_loss: 5.5508
Iteration: 8 of 25	train_loss: 5.3711
Iteration: 10 of 25	train_loss: 5.5116
Iteration: 12 of 25	train_loss: 5.5570
Iteration: 14 of 25	train_loss: 5.4498
Iteration: 16 of 25	train_loss: 5.4598
Iteration: 18 of 25	train_loss: 5.7475
Iteration: 20 of 25	train_loss: 5.7243
Iteration: 22 of 25	train_loss: 5.7819
Iteration: 24 of 25	train_loss: 5.6569
Iteration: 25 of 25	train_loss: 5.4232
Average Score for this Epoch: 5.556070804595947
-------------------- Epoch 26 of 100 --------------------
Iteration: 0 of 25	train_loss: 5.4921
Iteration: 2 of 25	train_loss: 5.3561
Iteration: 4 of 25	train_loss: 5.4273
Iteration: 6 of 25	train_loss: 5.3356
Iteration: 8 of 

Iteration: 4 of 25	train_loss: 5.0128
Iteration: 6 of 25	train_loss: 4.9527
Iteration: 8 of 25	train_loss: 4.9123
Iteration: 10 of 25	train_loss: 4.9572
Iteration: 12 of 25	train_loss: 5.0203
Iteration: 14 of 25	train_loss: 4.9150
Iteration: 16 of 25	train_loss: 5.0699
Iteration: 18 of 25	train_loss: 4.9307
Iteration: 20 of 25	train_loss: 4.9293
Iteration: 22 of 25	train_loss: 4.9229
Iteration: 24 of 25	train_loss: 4.9888
Iteration: 25 of 25	train_loss: 4.7644
Average Score for this Epoch: 4.929316520690918
--- new best score ---


-------------------- Epoch 38 of 100 --------------------
Iteration: 0 of 25	train_loss: 4.8404
Iteration: 2 of 25	train_loss: 4.6943
Iteration: 4 of 25	train_loss: 4.8881
Iteration: 6 of 25	train_loss: 4.7841
Iteration: 8 of 25	train_loss: 4.7800
Iteration: 10 of 25	train_loss: 4.7950
Iteration: 12 of 25	train_loss: 4.7946
Iteration: 14 of 25	train_loss: 4.8637
Iteration: 16 of 25	train_loss: 4.6701
Iteration: 18 of 25	train_loss: 4.7109
Iteration: 20 of 25

Iteration: 14 of 25	train_loss: 4.4021
Iteration: 16 of 25	train_loss: 4.3112
Iteration: 18 of 25	train_loss: 4.0312
Iteration: 20 of 25	train_loss: 4.5637
Iteration: 22 of 25	train_loss: 4.4845
Iteration: 24 of 25	train_loss: 4.4487
Iteration: 25 of 25	train_loss: 4.1643
Average Score for this Epoch: 4.291660785675049
--- new best score ---


-------------------- Epoch 50 of 100 --------------------
Iteration: 0 of 25	train_loss: 3.9346
Iteration: 2 of 25	train_loss: 4.4141
Iteration: 4 of 25	train_loss: 4.1096
Iteration: 6 of 25	train_loss: 4.2069
Iteration: 8 of 25	train_loss: 4.2656
Iteration: 10 of 25	train_loss: 4.2498
Iteration: 12 of 25	train_loss: 4.0562
Iteration: 14 of 25	train_loss: 4.2624
Iteration: 16 of 25	train_loss: 4.3706
Iteration: 18 of 25	train_loss: 4.5161
Iteration: 20 of 25	train_loss: 4.3931
Iteration: 22 of 25	train_loss: 4.3374
Iteration: 24 of 25	train_loss: 4.3344
Iteration: 25 of 25	train_loss: 3.8407
Average Score for this Epoch: 4.229165554046631
--- new

Iteration: 22 of 25	train_loss: 3.1969
Iteration: 24 of 25	train_loss: 3.5002
Iteration: 25 of 25	train_loss: 3.2334
Average Score for this Epoch: 3.3338382244110107
--- new best score ---


-------------------- Epoch 62 of 100 --------------------
Iteration: 0 of 25	train_loss: 3.1674
Iteration: 2 of 25	train_loss: 3.2408
Iteration: 4 of 25	train_loss: 3.2759
Iteration: 6 of 25	train_loss: 3.4973
Iteration: 8 of 25	train_loss: 3.2521
Iteration: 10 of 25	train_loss: 3.7471
Iteration: 12 of 25	train_loss: 3.3853
Iteration: 14 of 25	train_loss: 3.2535
Iteration: 16 of 25	train_loss: 3.5463
Iteration: 18 of 25	train_loss: 3.2441
Iteration: 20 of 25	train_loss: 3.4041
Iteration: 22 of 25	train_loss: 3.3574
Iteration: 24 of 25	train_loss: 3.1035
Iteration: 25 of 25	train_loss: 3.2415
Average Score for this Epoch: 3.337200164794922
-------------------- Epoch 63 of 100 --------------------
Iteration: 0 of 25	train_loss: 3.0084
Iteration: 2 of 25	train_loss: 3.1938
Iteration: 4 of 25	train_los

Iteration: 0 of 25	train_loss: 2.4005
Iteration: 2 of 25	train_loss: 2.5605
Iteration: 4 of 25	train_loss: 2.3597
Iteration: 6 of 25	train_loss: 2.5847
Iteration: 8 of 25	train_loss: 2.3315
Iteration: 10 of 25	train_loss: 2.4834
Iteration: 12 of 25	train_loss: 2.5769
Iteration: 14 of 25	train_loss: 2.7782
Iteration: 16 of 25	train_loss: 2.4192
Iteration: 18 of 25	train_loss: 2.7108
Iteration: 20 of 25	train_loss: 2.4252
Iteration: 22 of 25	train_loss: 2.6793
Iteration: 24 of 25	train_loss: 2.9232
Iteration: 25 of 25	train_loss: 2.1852
Average Score for this Epoch: 2.601419687271118
--- new best score ---


-------------------- Epoch 75 of 100 --------------------
Iteration: 0 of 25	train_loss: 2.2076
Iteration: 2 of 25	train_loss: 2.6565
Iteration: 4 of 25	train_loss: 2.9626
Iteration: 6 of 25	train_loss: 2.2233
Iteration: 8 of 25	train_loss: 2.5832
Iteration: 10 of 25	train_loss: 2.3722
Iteration: 12 of 25	train_loss: 2.4865
Iteration: 14 of 25	train_loss: 2.2434
Iteration: 16 of 25	t

Iteration: 10 of 25	train_loss: 1.8616
Iteration: 12 of 25	train_loss: 1.7148
Iteration: 14 of 25	train_loss: 1.8890
Iteration: 16 of 25	train_loss: 1.8197
Iteration: 18 of 25	train_loss: 1.9576
Iteration: 20 of 25	train_loss: 1.9452
Iteration: 22 of 25	train_loss: 2.0094
Iteration: 24 of 25	train_loss: 1.8809
Iteration: 25 of 25	train_loss: 1.6562
Average Score for this Epoch: 1.833349347114563
-------------------- Epoch 87 of 100 --------------------
Iteration: 0 of 25	train_loss: 1.6087
Iteration: 2 of 25	train_loss: 1.8297
Iteration: 4 of 25	train_loss: 1.5049
Iteration: 6 of 25	train_loss: 1.4667
Iteration: 8 of 25	train_loss: 1.8559
Iteration: 10 of 25	train_loss: 1.9592
Iteration: 12 of 25	train_loss: 1.9240
Iteration: 14 of 25	train_loss: 1.6489
Iteration: 16 of 25	train_loss: 1.8744
Iteration: 18 of 25	train_loss: 1.9550
Iteration: 20 of 25	train_loss: 2.5306
Iteration: 22 of 25	train_loss: 2.3455
Iteration: 24 of 25	train_loss: 1.7872
Iteration: 25 of 25	train_loss: 1.8012
Av

Iteration: 25 of 25	train_loss: 1.4846
Average Score for this Epoch: 1.3364571332931519
--- new best score ---


-------------------- Epoch 99 of 100 --------------------
Iteration: 0 of 25	train_loss: 1.6040
Iteration: 2 of 25	train_loss: 1.1928
Iteration: 4 of 25	train_loss: 1.0814
Iteration: 6 of 25	train_loss: 1.0155
Iteration: 8 of 25	train_loss: 1.1806
Iteration: 10 of 25	train_loss: 1.1292
Iteration: 12 of 25	train_loss: 1.3888
Iteration: 14 of 25	train_loss: 1.2122
Iteration: 16 of 25	train_loss: 1.2568
Iteration: 18 of 25	train_loss: 1.1606
Iteration: 20 of 25	train_loss: 1.1044
Iteration: 22 of 25	train_loss: 1.0373
Iteration: 24 of 25	train_loss: 1.1454
Iteration: 25 of 25	train_loss: 1.3817
Average Score for this Epoch: 1.2510466575622559
--- new best score ---


-------------------- Epoch 100 of 100 --------------------
Iteration: 0 of 25	train_loss: 1.2213
Iteration: 2 of 25	train_loss: 1.1314
Iteration: 4 of 25	train_loss: 1.3588
Iteration: 6 of 25	train_loss: 1.0004
Ite

### Inference
Now we can use our trained model to create summaries. Here we are clearly overfitting, as we only trained on 700 examples. (i.e. the model does not generalize at all.)


In [59]:
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   './models/headlines/my_model',
                                   'INFER',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   batch_size = len(converted_texts[:50]),
                                   clip = clip,
                                   keep_probability = 1.0,
                                   learning_rate = 0.0,
                                   beam_width = 5,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   inference_targets = False,
                                   pretrained_embeddings_path = pretrained_embeddings_path)

summarizer.build_graph()
preds = summarizer.infer(converted_texts[:50],
                         restore_path =  './models/headlines/my_model',
                         targets = converted_summaries[:50])




Loaded pretrained embeddings.
Graph built.
INFO:tensorflow:Restoring parameters from ./models/headlines/my_model


I0331 19:20:43.667531  3056 saver.py:1270] Restoring parameters from ./models/headlines/my_model


Done.


In [60]:
# show results
summarizer_model_utils.sample_results(preds,
                                      ind2word,
                                      word2ind,
                                      converted_summaries[:50],
                                      converted_texts[:50])




 ----------------------------------------------------------------------------------------------------
Actual Text:
london queen elizabeth ii who has been battling a cold for more than a week missed a new year s day church service at her country estate in sandringham buckingham palace said on sunday a week earlier the queen who is 90 missed a christmas day church service for the first time since 1988 because of the illness the queen does not yet feel ready to attend church as she is still recuperating from a heavy cold the palace said in a statement the queen s husband prince philip who had also been ill was well enough to attend both services in the church at sandringham which is in norfolk on the east coast of england the queen who <UNK> to the throne in 1952 became the world s monarch following the death of king bhumibol adulyadej of thailand in october she is also britain s monarch having last year surpassed queen victoria s reign her mother lived until the age of 101

Actual Sum